# TChecker

## 1. 摘要

PHP应用为终端用户在 Web 上进行交互提供了各种接口，因此它们很容易出现污点型漏洞，例如 SQL 注入和跨站脚本攻击。由于其高效率，静态分析被广泛使用在应用程序部署之前检测污点型漏洞。不幸的是，因为PHP语言的高度复杂性，实现精确的静态污点分析很困难。现有的污点分析方法因不全面的过程间分析和各种实现问题，导致较高的误报和漏报。

作者提出了 <a href="https://github.com/cuhk-seclab/TChecker" title="A static taint analysis tool for PHP applications.">TChecker</a>，这是一种上下文敏感的过程间静态污点分析工具，用于检测PHP应用中的污点型漏洞。作者发现，支持对象和类型系统对于静态分析PHP程序至关重要，因此他们首先在 TChecker 中仔细地对PHP对象和相关的面向对象编程特性进行建模。然后，它对PHP对象迭代地执行过程间数据流分析，以细化对象类型，从而可以精确识别调用目标。作者还付出大量努力来支持PHP的其他动态特性，例如动态包含。

作者在多个现代PHP应用上全面评估了 TChecker，并证明了其在漏洞检测方面的高效性。具体来说，TChecker 在这些PHP应用中成功检测到 18 个先前未知的漏洞，并分配了两个 CVE（CVE-2022-35212 和 CVE-2022-35213）。作者将 TChecker 与相关的静态分析工具进行比较，发现它在检测漏洞方面明显优于它们。TChecker 还可以以相对较高的精度发现现有工具检测的所有漏洞。最后，作者公开了原型实现的源代码，以方便未来的研究。


## 2. 前提

### 2.1 PHP-CS-Fixer：通过 PHP-CS-Fixer 来修复编码规范

<a href="https://github.com/PHP-CS-Fixer/PHP-CS-Fixer" title="A tool to automatically fix PHP Coding Standards issues.">PHP CS Fixer</a> (PHP代码标准修复工具) 可以自动修复代码，使其符合特定的编码规范，无论是 PSR-1、PSR-2 等PHP编码标准，还是其他由社区驱动的标准，如 Symfony 编码规范。它甚至还可以通过配置文件定义自己或团队的编码风格。这有助于确保代码在整个项目中保持一致，易于阅读和维护。

### 2.2 phpjoern：为PHP应用生成节点文件和边文件

<a href="https://github.com/malteskoruppa/phpjoern" title="Parser utility to generate ASTs from PHP source code suitable to be processed by Joern.">phpjoern</a> 是一个用于 Joern 的实用工具。它利用 php-ast 扩展将PHP项目代码生成 ASTs (抽象语法树），并将这些 ASTs 导出到可被 Joern 解析的 CSV 文件中，以便后续可以通过 Joern 进行代码安全分析和漏洞检测。

## 3. 动机和挑战

### 3.1 动机

当用户提交的的数据（例如污点）未经过充分清理，并用于应用程序的关键操作（例如 sinks）时，就会出现安全漏洞，这类漏洞称为污点型漏洞。污点分析是在实践中发现污点型漏洞的事实上的方法，它跟踪程序执行过程中源自外部源（例如不可信用户提交的数据）的污点传播，检查受污染的数据是否流向程序的关键位置（sinks），最后报告可能被攻击者操控的 sinks。

污点分析一般分为动态和静态两种方式。动态方式通常注入特殊的 payloads，并以黑盒方式检查它们在部署的 Web 应用上的复现情况。然而，由于现代 Web 应用的复杂性，这类方法只能覆盖并检测应用的一小部分，应用的其余部分未能检查，因此无法发现其中的漏洞。此外，由于需要手动部署和配置应用程序，动态方式一般不可扩展。静态方式通过分析源代码来报告潜在的污点型漏洞，可以实现高的代码覆盖率，并且是更高效和可扩展的。

**PHP程序静态分析**。执行过程间程序分析对于追踪变量的数据流非常重要，而分析调用关系是过程间程序分析的前提。PHP具有弱类型、动态类型和自动加载外部类和接口的特点，使得精确地推理出PHP程序的调用关系更具有挑战性。尽管数据流分析是一个基本技术，但由于PHP代码的动态性，PHP程序的数据流分析相比其他语言有新的挑战。

由于调用图和数据流分析是静态污点分析的重要组成部分，因此广大研究员为PHP程序分析提出了不同的设计，作者将关键思想总结如下：

**函数名的（部分）匹配**。分析调用关系对于跟踪跨函数传播的污点是必要的。为了确定方法调用的目标，一种常见的做法是忽略调用点的对象，仅匹配被调用的方法名。RIPS 将被调用的方法名 f() 与整个PHP程序中的函数定义比较，并将匹配的视为调用点 $obj->f() 的目标函数，但这种方法存在很多误报。相反，PHPJoern 仅考虑具有唯一被调函数名的函数调用。然而，这会产生高漏报率。

**数据流分析**。解决调用关系的高级方式是利用数据流分析。RIPS 的专有版本（RIPS-A）是对PHP程序进行数据流分析的最先进的工具。然而由于缺乏过程间分析，RIP-A 无法推断在其它函数作用域中分配的变量（例如函数参数）的类型或值。因此，它无法支持PHP应用中的一些常见的调用模式。此外，它不是开源的，研究人员必须自己实现静态分析技术（例如调用图），或者在 PHPJoern 等开源工具上实现。

**函数摘要**。执行过程间数据流分析需要分析调用点的目标函数，为了避免重复分析，创建函数摘要以总结函数内的数据流。然而，先前的工作假设每个调用点在任何上下文中调用同一个函数，为了性能而牺牲了分析精度。

### 3.2 挑战

作者的目标是改进现有的静态分析技术，通过开发一个综合的过程间静态污点分析工具，以检测PHP应用中的污点型漏洞。

- 首先，调用关系推断和过程间数据流分析非常困难。
- 此外，需要提出一种新的过程间污点跟踪算法。它应该精确的——污点只传播到当前调用上下文的目标函数；并且高效——即使调用点可能执行多次，也能有效地确定调用上下文中调用点的目标函数。
- 最后，需要对PHP程序中一些复杂但常用的功能（例如动态包含和对象属性）进行建模，以解决现有工具的不足。众所周知，对PHP中的动态特性进行建模是很困难的，但它使我们能够理解更多的PHP操作的语义，并发现应用程序深处存在的漏洞。


## 4. 概述

作者提出了TChecker，这是一种精确的上下文敏感的过程间静态污点分析工具。它支持面向对象的特性，可以识别PHP应用中的污点型漏洞。

进行过程间污点分析对于检测PHP应用中的漏洞非常重要，它的主要挑战是分析调用关系。为此，TChecker 首先构建一个调用图，这有助于后续函数调用时执行更准确的污点传播。在污点传播前构建调用图有两重好处，首先构建调用图可以以上下文敏感的方式包含调用点所有可能的目标函数。因此，给定一个特定的调用上下文，TChecker 可以直接从可能的目标函数中正确地选出调用点的目标函数。在不提前构建调用图的情况下，TChecker 在污点分析时每次遇到调用点（即使是在相同的上下文）都必须重复推断调用点的调用目标。其次，调用图允许 TChecker 进行有选择性的污点分析，这可以避免对不相关函数进行不必要的分析，从而提高整体分析效率。

TChecker 的架构如图 1 所示。给定PHP应用的源代码，TChecker 通过推断每个调用点的调用目标来逐步地构建调用图；然后它对整个程序进行上下文敏感的过程间污点分析，从而识别污点型漏洞。

![TChecker-architecture-1](imgs/TChecker-architecture-1.jpg)

<p style="text-align: center;">图1: TChecker的总体架构</p>

## 5. 数据集和设置

作者挑选了17个PHP应用作为评估数据集，这些应用的代码超过 1 百万行。它们位于表 1 的第一列。评估数据集由两类应用组成。第一类是流行且大众熟知的PHP应用，例如 WordPress、MediaWiki 和 Joomla。使用 <a href="https://dl.acm.org/doi/fullHtml/10.1145/3442381.3449826" title="LChecker: Detecting Loose Comparison Bugs in PHP">LChecker</a> 中相同的标准，作者选择 9 个市场份额超过 0.1% 或是在 GitHub 上拥有超过 2k 颗星的PHP应用，并在表 1 用上标进行标记。第二类是相对较不流行、先前工作已评估的应用，作者完整地收录了此类应用。

作者将 TChecker 与 用于PHP应用漏洞检测的高级静态分析工具 PHPJoern 和 RIPS 的最新开源版本进行比较。而其余的污点分析工具要么不开源，要么代码不完整，因此无法和它们进行比较。最后，为了进行公平比较，作者对 TChecker、RIPS 和 PHPJoern 采用了相同的源、sinks 和清洗规则的定义。

<p style="text-align: center;">表1: 漏洞检测的评估结果</p>

![TChecker-evaluation_results_of_vulnerability_detection-2](imgs/TChecker-evaluation_results_of_vulnerability_detection-2.jpg)

其中，TP 表示真阳性；FP表示误报；上标 T, J, R 表示 TChecker、PHPJoern 和 RIPS 的结果。UP 只表示 TChecker 发现的漏洞数量。UN 只表示 PHPJoern 和 RIPS 发现的漏洞数量。NP 表示 TChecker 发现的新漏洞数量。'-' 表示作者未能将工具应用于该程序。

## 6. 讨论和相关工作

### 6.1 讨论

- PHP静态分析

TChecker 旨在对PHP应用中复杂特性进行建模，然而它仍然无法完全支持一些PHP特性，包括动态数组、可变函数等。TChecker在评估时存在一些误报。未来可以在 TChecker 上开发新的算法和实现，以进一步改进静态技术。此外，研究不同场景下简化模型对不同特征的影响会很有趣。在这项工作中，作者还展示了上下文敏感的过程间分析对检测PHP应用中污点型漏洞的重要性。

- 调用图

精确的调用图可以帮助发现PHP应用中更多的污点型漏洞。然而，因为分析数据库语句与这项工作无关，TChecker 目前不考虑二次注入。此外，静态分析可以与动态方法一起使用，如 <a href="https://www.usenix.org/conference/usenixsecurity18/presentation/alhuzali" title="NAVEX: Precise and Scalable Exploit Generation for Dynamic Web Applications">NAVEX</a> 中所示。

- 自动验证

静态分析通常会在漏洞检测时出现误报。Navex 已经证明使用符号执行来进行自动漏洞验证的可行性。然而，由于发布的代码不完整，作者无法将其应用到这项工作中。作者计划未来实现符号执行来验证和利用 TChecker 发现的漏洞。但是，自动验证 TChecker 发现的漏洞很困难，除了在PHP应用实现符号执行算法之外，还需要解决额外的难题，例如传统的符号引擎只考虑调用点的一个目标函数。

### 6.2 相关工作

- PHP应用的漏洞检测

近几年里，PHP漏洞的检测引起了人们广泛关注。先前关于静态安全分析的研究侧重于数据流（污点）分析，来识别跨站脚本、SQL 注入、拒绝服务等漏洞。TChecker 提出了几种通用技术，如精确过程间数据流分析和对象属性处理，以改进PHP最先进的静态分析。事实证明，这些技术在精度和扩展性方面带来了许多提升。尽管目前只在PHP实现了它们，但作者相信可以扩展到其他语言，以进一步促进静态程序分析领域的发展。

另一条研究主线是利用动态扫描方法来检测PHP中的漏洞。此类工具驱动PHP代码的具体执行，因此可以真正支持PHP语言的动态性。然而，像状态间依赖关系这样的关键问题使得动态扫描工具更难有效地达到深层代码。像 Navex 和 Chainsaw 这样的混合方法应用符号执行对状态间依赖关系进行建模，可以生成种子，以引导动态扫描器在代码深处找到隐藏的漏洞。类似地，作者对调用图的细化可以改进静态符号执行。

- 调用图分析

调用图分析是一种基础的程序分析技术，用于各种任务，例如漏洞检测、控制流完整性等。调用图分析不仅是PHP，也是其他编程语言的重要课题。Yamaguchi 等人将调用图集成到代码属性图，以对 C/C++ 程序中常见漏洞进行建模，并通过过程间分析进一步扩展它。最近，Lu 和 Hu 基于结构类型和指针分析，通过对 C/C++ 程序的多层类型分析来细化间接调用的目标。然而与其他语言不同，PHP作为一种弱类型、动态类型的语言，缺少必要的变量类型的信息，使得执行调用图分析变得更加困难。TChecker 解决了这些PHP特有的难题，并实现了精确的调用图分析。